In [6]:
import os
import sys
import gc
import random
import datetime
import time

import random
import pandas as pd
import numpy as np
import xgboost as xgb
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

seed = 3
random.seed(seed)
np.random.seed(seed)

### Outlier filter

### Extract features

1. Wifi
  * Wifi Signal Power Matrix ### param: drop_num, drop_rate
  * Wifi Signal Connect Wifi-Id ### fill NA with 'Null'
2. Time
  * Weekday & Hourmap ### Param: one_hot
3. Location
  * longditude
  * latitude
4. Time & Category
  * Weekday-Category shopping_num rate
  * hourmap-Category shopping_num rate
5. Time & Price
  * Weekday-Hourmap shopping 0% price
  * Weekday-Hourmap shopping 25% price
  * Weekday-Hourmap shopping 50% price
  * Weekday-Hourmap shopping 75% price
  * Weekday-Hourmap shopping 100% price
  * Weekday-Hourmap shopping price std
6. User & Shop
  * User last shopping shop_id ### before sample time
  * User most likely shopping shop_id ### before sample time
7. User & Category
  * User last shopping category ### before sample time
  * User most shopping category ### before sample time
8. User & Price
  * User shopping mean price ### before sample time
  * User shopping price std ### before sample time

In [4]:
path = '/home/zhouzr/data/user-location-forcast/'
os.chdir(path)
train = pd.read_csv('./data/train_local.csv')
test = pd.read_csv('./data/test_local.csv')
shop = pd.read_csv('./original_data/训练数据-ccf_first_round_shop_info.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/home/zhouzr/data/user-location-forcast/'

In [ ]:
mall = shop.mall[5]
train = train.merge

### outlier filter

In [2]:
def location_outlier_filter(df, filter_rate=0.005):
    def outlier_filter(df, col, min_filter, max_filter):
            min_outlier = np.percentile(df[col], min_filter*100)
            max_outlier = np.percentile(df[col], max_filter*100)
            outlier_index = df[(df[col] > max_outlier) | (df[col] < min_outlier)].index.tolist()
            return outlier_index
    idx1 = outlier_filter(df, 'latitude', filter_rate, filter_rate)
    idx2 = outlier_filter(df, 'longitude', filter_rate, filter_rate)
    drop_idx = list(set(idx1 + idx2))
    return df.drop(drop_idx, axis=0).reset_index(drop=True)

def user_behavior_outlier_filter(df, filter_num=40):
    user_behavior = df.groupby('user_id')['sample_id'].count()
    drop_idx = user_behavior[user_behavior>40].index.tolist()
    return df.drop(drop_idx, axis=0).reset_index(drop=True)

### wifi feat

In [7]:
def wifi_connect(df_train, df_test, drop_num):
    wifi_connect_list = {}
    wifi_count = {}
    wifi_infos = df_train['wifi_infos']
    for wifi_infos_i in wifi_infos:
        wifi_infos_i = [wifi.split('|') for wifi in wifi_infos_i.split(';')]
        for wifi in wifi_infos_i:
            if wifi[2] != 'false':
                if wifi[0] in wifi_count:
                    wifi_count[wifi[0]] += 1
                else:
                    wifi_count[wifi[0]] = 1
    for k, v in wifi_count.items():
        if v > drop_num:
            wifi_connect_list[k] = 0
    def extract_wifi_connect(df, wifi_connect_list):
        result = []
        wifi_infos = df['wifi_infos']
        for wifi_infos_i in wifi_infos:
            wifi_infos_i = [wifi.split('|') for wifi in wifi_infos_i.split(';')]
            result_i = wifi_connect_list.copy()
            for wifi in wifi_infos_i:
                if wifi[2] != 'false':
                    if wifi[0] in wifi_connect_list:
                        result_i[wifi[0]] = 1
            result.append(result_i)
        result = pd.DataFrame(result)
        return result
    train_wifi_connect = extract_wifi_connect(df_train, wifi_connect_list)
    test_wifi_connect = extract_wifi_connect(df_test, wifi_connect_list)
    feat_cols = ['wifi_connect'] * train_wifi_connect.shape[1]
    return train_wifi_connect, test_wifi_connect, feat_cols

def wifi_power(df_train, df_test, drop_num):
    wifi_power_list = {}
    wifi_infos = df_train['wifi_infos']
    wifi_count = {}
    for wifi_infos_i in wifi_infos:
        wifi_infos_i = [wifi.split('|') for wifi in wifi_infos_i.split(';')]
        for wifi in wifi_infos_i:
            if wifi[0] in wifi_count:
                wifi_count[wifi[0]] += 1
            else:
                wifi_count[wifi[0]] = 1
    for k, v in wifi_count.items():
        if v > drop_num:
            wifi_power_list[k] = 0
    def extract_wifi_power(df, wifi_power_list):
        wifi_infos = df.wifi_infos
        result = []
        for wifi_infos_i in wifi_infos:
            wifi_infos_i = [wifi.split('|') for wifi in wifi_infos_i.split(';')]
            result_i = wifi_power_list.copy()
            for wifi in wifi_infos_i:
                if wifi[0] in wifi_power_list:
                    result_i[wifi[0]] = float(wifi[1]) + 120
            result.append(result_i)
        result = pd.DataFrame(result)
        return result
    train_wifi_power = extract_wifi_power(df_train, wifi_power_list)
    test_wifi_power = extract_wifi_power(df_test, wifi_connect_list)
    feat_cols = ['wifi_power'] * train_wifi_power.shape[1]
    return train_wifi_power, test_wifi_power, feat_cols

### time

In [78]:
def hourmap(t):
    if (t >= datetime.time(22,0)) or (t < datetime.time(8,0)):
        return 0
    if (t >= datetime.time(8,0)) and (t < datetime.time(9,30)):
        return 1
    if (t >= datetime.time(9,30)) and (t < datetime.time(11,30)):
        return 2
    if (t >= datetime.time(11,30)) and (t < datetime.time(14,0)):
        return 3
    if (t >= datetime.time(14,0)) and (t < datetime.time(17,0)):
        return 4
    if (t >= datetime.time(17,0)) and (t < datetime.time(20,0)):
        return 5
    if (t >= datetime.time(20,0)) and (t < datetime.time(22,0)):
        return 6

def weekday(df_train, df_test):
    onehot = OneHotEncoder()
    train_weekday = pd.to_datetime(df_train.time_stamp).dt.weekday.values.reshape(-1, 1)
    test_weekday = pd.to_datetime(df_test.time_stamp).dt.weekday.values.reshape(-1, 1)
    train_weekday = onehot.fit_transform(train_weekday)
    test_weekday = onehot.transform(test_weekday)
    return train_weekday, test_weekday

In [ ]:

        class UserFeatExt(object):
        
    def extract_user_id(self, df_):
        df = df_.copy()
        user_cnt = df.groupby('user_id')['user_id'].count()
        user_set = set(df[df.shop_id.isnull()].user_id.values) |\
        set(df[df.shop_id.notnull()].user_id.values) |\
        set(user_cnt[user_cnt>5].index.tolist())
        
        df.loc[-df.user_id.isin(user_set),'user_id'] = 'Null'
        le = LabelEncoder()
        result = le.fit_transform(df.user_id.values).reshape(-1,1)
        return result

    
class SpaceFeat(object):
    
    def fit(self, df):
        self.scaler = MinMaxScaler()
        self.scaler.fit(df[['latitude','longitude']])
        
    def extract_space(self, df):
        result = self.scaler.transform(df[['latitude','longitude']])
        return result
    
    def extract_space_dist(self, df, shop):
        def squaer_dist(loc1, loc2_list):
            dist = np.power(np.sum(np.power(loc2_list-loc1,2),1),0.5)
            return dist
        df_space = df[['latitude','longitude']].values
        shop_loc = shop[['latitude','longitude']].values
        feat = np.apply_along_axis(squaer_dist, 1, df_space, **{'loc2_list':shop_loc})
        return feat


class TimeFeat(object):
    
    def extract_weekday(self, df, onehot=True):
        result = df.time_stamp.dt.weekday.values.reshape(df.shape[0], 1)
        if onehot:
            result = pd.get_dummies(pd.Series(result.reshape(-1))).values
        return result
    
    def extract_hour(self, df, onehot=True):
        result = df.time_stamp.dt.hour.values.reshape(df.shape[0], 1)
        if onehot:
            result = pd.get_dummies(pd.Series(result.reshape(-1))).values
        return result
    
    def extract_hour_map(self, df, onehot=True):
        
        result = df.time_stamp.dt.time.apply(time_map).values.reshape(df.shape[0], 1)
        if onehot:
            result = pd.get_dummies(pd.Series(result.reshape(-1))).values
        return result
    
    def extract_weekday_hour_map(self, df, onehot=True):
        weekday = self.extract_weekday(df, onehot=False)
        hour_map = self.extract_hour_map(df, onehot=False)
        result = weekday * 100 + hour_map
        if onehot:
            result = pd.get_dummies(pd.Series(result.reshape(-1))).values
        return result